<a href="https://github.com/xuehangcang/DeepLearning/blob/main/docs/PyTorch/%E5%9F%BA%E7%A1%80%E7%9F%A5%E8%AF%86/1.%E5%BF%AB%E9%80%9F%E5%85%A5%E9%97%A8.ipynb" download=""><svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" width="110" height="20" role="img" aria-label="jupyter: notebook"><title>jupyter: notebook</title><linearGradient id="s" x2="0" y2="100%"><stop offset="0" stop-color="#bbb" stop-opacity=".1"/><stop offset="1" stop-opacity=".1"/></linearGradient><clipPath id="r"><rect width="110" height="20" rx="3" fill="#fff"/></clipPath><g clip-path="url(#r)"><rect width="49" height="20" fill="#555"/><rect x="49" width="61" height="20" fill="#fe7d37"/><rect width="110" height="20" fill="url(#s)"/></g><g fill="#fff" text-anchor="middle" font-family="Verdana,Geneva,DejaVu Sans,sans-serif" text-rendering="geometricPrecision" font-size="110"><text aria-hidden="true" x="255" y="150" fill="#010101" fill-opacity=".3" transform="scale(.1)" textLength="390">jupyter</text><text x="255" y="140" transform="scale(.1)" fill="#fff" textLength="390">jupyter</text><text aria-hidden="true" x="785" y="150" fill="#010101" fill-opacity=".3" transform="scale(.1)" textLength="510">notebook</text><text x="785" y="140" transform="scale(.1)" fill="#fff" textLength="510">notebook</text></g></svg></a>

## 1.1 基础知识  

大多数机器学习工作流都涉及到处理数据、创建模型、优化模型参数以及保存训练好的模型。本教程介绍了如何在PyTorch中实现完整的机器学习工作流，并提供了相关概念的学习链接。  

我们将使用FashionMNIST数据集来训练一个神经网络，该网络可以预测输入图像属于以下哪个类别：T恤/上衣、裤子、套衫、连衣裙、外套、凉鞋、衬衫、运动鞋、包或踝靴。  

本教程假设你已经基础了解Python和深度学习的概念。

## 1.2 处理数据  

PyTorch有两个与数据处理相关的基本元素：`torch.utils.data.DataLoader` 和 `torch.utils.data.Dataset`。  
`Dataset` 存储样本及其对应的标签，而 `DataLoader` 则在 `Dataset` 周围建立一个可迭代的包装器。

In [3]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

PyTorch提供了特定领域的库，例如TorchText、TorchVision和TorchAudio，它们都包含数据集。在本教程中，我们将使用一个TorchVision数据集。  

torchvision.datasets模块包含许多现实世界的视觉数据集，如CIFAR、COCO。在本教程中，我们将使用FashionMNIST数据集。每个TorchVision数据集都包括两个参数：transform和target_transform，分别用于修改样本和标签。

In [4]:
# 从开放数据集中下载训练数据
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# 从开放数据集中下载测试数据
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

100.0%


Extracting data\FashionMNIST\raw\train-images-idx3-ubyte.gz to data\FashionMNIST\raw



100.0%


Extracting data\FashionMNIST\raw\train-labels-idx1-ubyte.gz to data\FashionMNIST\raw



100.0%


Extracting data\FashionMNIST\raw\t10k-images-idx3-ubyte.gz to data\FashionMNIST\raw



100.0%

Extracting data\FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to data\FashionMNIST\raw



我们将`Dataset`作为参数传递给`DataLoader`。这将包装我们的数据集为一个迭代器，并支持自动批处理、采样、洗牌和多进程数据加载。在这里，我们定义批次大小为64，即数据加载器迭代器中的每个元素将返回 64 个特征和标签的批次。

In [5]:
batch_size = 64

# 创建数据加载器
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


## 1.3 创建模型  

在PyTorch中定义神经网络，我们创建一个继承自`nn.Module`的类。  

我们在`__init__`函数中定义网络的层，并在`forward`函数中指定数据如何通过网络。为了加速神经网络的操作，我们将其移动到GPU或MPS（如果有）。  

获取用于训练的CPU、GPU或MPS设备。

In [6]:
# 选择CPU、GPU或MPS设备进行训练
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

# 定义模型
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


## 1.4 优化模型参数  

要训练一个模型，我们需要一个`损失函数`和一个`优化器`。

In [7]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

在单个训练循环中，模型对训练数据集进行预测（以批次形式提供），并通过反向传播预测误差来调整模型的参数。

In [8]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # 计算预测误差
        pred = model(X)
        loss = loss_fn(pred, y)

        # 反向传播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

我们还会对模型在测试数据集上的表现进行检查，以确保其学习效果。

In [9]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

训练过程会进行多轮迭代（epoch）。在每轮迭代中，模型会学习参数以提高预测准确性。我们会在每轮迭代结束后打印模型的准确率和损失值，并希望看到准确率逐步提高，损失值逐步减小。

In [10]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.311305  [   64/60000]
loss: 2.296385  [ 6464/60000]
loss: 2.277378  [12864/60000]
loss: 2.266208  [19264/60000]
loss: 2.256758  [25664/60000]
loss: 2.226165  [32064/60000]
loss: 2.229622  [38464/60000]
loss: 2.204165  [44864/60000]
loss: 2.206017  [51264/60000]
loss: 2.160655  [57664/60000]
Test Error: 
 Accuracy: 44.9%, Avg loss: 2.163659 

Epoch 2
-------------------------------
loss: 2.182999  [   64/60000]
loss: 2.172259  [ 6464/60000]
loss: 2.123006  [12864/60000]
loss: 2.131096  [19264/60000]
loss: 2.080843  [25664/60000]
loss: 2.032379  [32064/60000]
loss: 2.048243  [38464/60000]
loss: 1.988958  [44864/60000]
loss: 2.000666  [51264/60000]
loss: 1.914852  [57664/60000]
Test Error: 
 Accuracy: 55.7%, Avg loss: 1.919090 

Epoch 3
-------------------------------
loss: 1.964226  [   64/60000]
loss: 1.931629  [ 6464/60000]
loss: 1.827652  [12864/60000]
loss: 1.851110  [19264/60000]
loss: 1.735007  [25664/60000]
loss: 1.699753  [32064/600

##  1.5 模型保存  

保存模型的常见方法是将内部状态字典（包含模型参数）序列化。

In [11]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


## 1.6 模型加载  

加载模型的过程包括重新创建模型结构并将状态字典加载到其中。

In [12]:
model = NeuralNetwork()
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

现在可以使用这个模型进行预测了。

In [13]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
